ERP is an acronym that stands for **Enterprise Resource Planning** (ERP). 

It's a business process management software that manages and integrates a company's financials, supply chain, operations, commerce, reporting, manufacturing, and human resource activities. 

One important ERP entity is the **Account Receivable (AR)**: it refers to the money a company's customers owe for goods or services they have received.

 **Account Receivable (AR)** could be:
 * Invoice
 * Credit Note
 * Debit Note
 * Cancellation
 * Miscellaneous

Each AR is made by several part like the **header** – the part with general information about customers/suppliers that define the invoice – the **list of items**, the **list of payments**, details about the **customers**, details about the **shipping**, ...

# Parameters

* N: number of invoices
* M: number of payments
* K: number of customers

In [5]:
N=10000
M=12500
K=150

# AR Header

The Header of an AR document contains some general information like
* Customer ID
* Value
* Due Date
* Posting Date
* Document Number - must be unique per fiscal year
* Fiscal Year
* Document Type

Assumptions:
* we have "Invoice" has only type

In [6]:
from random import randint
from datetime import datetime,timedelta
def headerGenerator(k=5):
  postingDate = datetime(2022,1,1)+timedelta(randint(0,200))
  return {
          "customerId":"Customer_{customerId}".format(customerId=str(randint(0,k)+1).zfill(3)),
          "value":randint(50,10000),
          "documentCurrency":"EUR", 
          "postingDate":postingDate.strftime("%Y-%m-%d"),
          "dueDate":(postingDate+timedelta(randint(0,60))).strftime("%Y-%m-%d"),
          "fiscalYear":postingDate.strftime("%Y"),
          "documentType":"Invoice"
         }


def headerList(k=5,n=1000):
  rawHeaderList = [headerGenerator(k) for k in range(n)]
  rawHeaderList.sort(key=lambda row: row.get("postingDate"))
  for pos,val in enumerate(rawHeaderList):
    val["documentNumber"]="2022-{docNum}".format(docNum=str(pos).zfill(5))
  return rawHeaderList
  
myARList = headerList(K,N)

# AR Payments

List of lines that represent a payment made by a customer on a given AR.
* Document Number
* Payment Date
* Value Paid

In [7]:
def paymentGenerator(InvoiceList):
  documentNumber = "2022-{docNum}".format(docNum=str(randint(0,len(InvoiceList)-1)).zfill(5))
  invoice = [k for k in InvoiceList if k.get("documentNumber")==documentNumber][0]
  postingDate = datetime.strptime(invoice.get("postingDate"),"%Y-%m-%d")
  return { 
          "documentNumber":documentNumber,
          "paymentDate":(postingDate+timedelta(randint(15,90))).strftime("%Y-%m-%d"),
          "valuePaid":randint(1,invoice.get("value"))
          ,"documentCurrency":invoice.get("documentCurrency")
         }


def paymentList(InvoiceList,m=250):
  return [paymentGenerator(InvoiceList) for k in range(m)]
   
myPaymentList = paymentList(myARList,M)


# Part 00
* Define the type of each table (Log or Registry): which are the keys of these tables?

Both Header and Payments are log, because I cannot update or delete no one of them.
Keys:
* Header: documentNumber and fiscalYear
* Payments: documentNumber and paymenteDate (under the assumpions: i) I can receive multiple payments for a given AR, ii) I cannot receive more than one payment for a given invoice per day

# Part 01
* Create the two RDDs checking everything is ok!
* Create a unique RDD with pieces of information both from header and payments

In [8]:
headerRDD = sc.parallelize(myARList)
paymentsRDD = sc.parallelize(myPaymentList)
headerRDD.count()==N,paymentsRDD.count()==M

(True, True)

Spark is based on piping operation using '.' to concatenate them...

In [10]:
def filterRow(row):
  return row.get("customerId")=='Customer_042'
headerRDD.filter(lambda headerDict: filterRow(headerDict)).map(lambda valueRow: valueRow.get("value")).filter(lambda row: row>1000).count()

4

In [11]:
semiJoinHeaderRDD = headerRDD.map(lambda row:((row.get("documentNumber"), row.get("fiscalYear")),row))
semiJoinHeaderRDD.first()

(('2022-00000', '2022'),
 {'customerId': 'Customer_043',
  'value': 902,
  'documentCurrency': 'EUR',
  'postingDate': '2022-01-01',
  'dueDate': '2022-01-12',
  'fiscalYear': '2022',
  'documentType': 'Invoice',
  'documentNumber': '2022-00000'})

In [12]:
semiJoinPaymentsRDD = paymentsRDD.map(lambda row:((row.get("documentNumber"), datetime.strptime(row.get("paymentDate"),"%Y-%m-%d").strftime("%Y")),row))
semiJoinPaymentsRDD.first()

(('2022-01464', '2022'),
 {'documentNumber': '2022-01464',
  'paymentDate': '2022-04-12',
  'valuePaid': 6543,
  'documentCurrency': 'EUR'})

In [15]:
# semiJoinPaymentsRDD.join(semiJoinHeaderRDD)
# semiJoinPaymentsRDD.join(headerRDD)

# Join format
Join result is a tuple of two (key,(values)) while Values is a tuple of two (valueLeft,valueRight):
* joinResult[0]: key
* joinResult[1][0]: leftValue
* joinResult[1][1]: rightValue

In [18]:
joinedRdd = semiJoinPaymentsRDD.join(semiJoinHeaderRDD)
joinFirstResult = joinedRdd.first()
print("Keys:{keys}".format(keys=joinFirstResult[0]))
print("Left part of the join:{valueLeft}".format(valueLeft=joinFirstResult[1][0]))
print("Right part of the join:{valueRight}".format(valueRight=joinFirstResult[1][1]))

Keys:('2022-04514', '2022')
Left part of the join:{'documentNumber': '2022-04514', 'paymentDate': '2022-06-26', 'valuePaid': 1560, 'documentCurrency': 'EUR'}
Right part of the join:{'customerId': 'Customer_7503', 'value': 1993, 'documentCurrency': 'EUR', 'postingDate': '2022-04-01', 'dueDate': '2022-05-23', 'fiscalYear': '2022', 'documentType': 'Invoice', 'documentNumber': '2022-04514'}


In [23]:
def formatRow(row):
  basicRow = {"header":row[1][1]} #header
  basicRow["keyTuple"] = row[0]
  basicRow["paymentList"] = [row[1][0]]
  return basicRow
niceJoineRDD = joinedRdd.map(lambda row: formatRow(row))
niceJoineRDD.first()

{'header': {'customerId': 'Customer_7503',
  'value': 1993,
  'documentCurrency': 'EUR',
  'postingDate': '2022-04-01',
  'dueDate': '2022-05-23',
  'fiscalYear': '2022',
  'documentType': 'Invoice',
  'documentNumber': '2022-04514'},
 'keyTuple': ('2022-04514', '2022'),
 'paymentList': [{'documentNumber': '2022-04514',
   'paymentDate': '2022-06-26',
   'valuePaid': 1560,
   'documentCurrency': 'EUR'}]}

In [26]:
niceJoineRDD.map(lambda x: (x.get("keyTuple"),1)).reduceByKey(lambda left,right: left+right).first()

(('2022-04514', '2022'), 1)

In [38]:
result = niceJoineRDD.filter(lambda row: row.get("keyTuple")==('2022-09741', '2022')).collect()
for pos,val in enumerate(result):
    print("Element number {pos}".format(pos=pos))
    print(val)

Element number 0
{'header': {'customerId': 'Customer_2351', 'value': 3930, 'documentCurrency': 'EUR', 'postingDate': '2022-07-15', 'dueDate': '2022-08-03', 'fiscalYear': '2022', 'documentType': 'Invoice', 'documentNumber': '2022-09741'}, 'keyTuple': ('2022-09741', '2022'), 'paymentList': [{'documentNumber': '2022-09741', 'paymentDate': '2022-09-14', 'valuePaid': 882, 'documentCurrency': 'EUR'}]}
Element number 1
{'header': {'customerId': 'Customer_2351', 'value': 3930, 'documentCurrency': 'EUR', 'postingDate': '2022-07-15', 'dueDate': '2022-08-03', 'fiscalYear': '2022', 'documentType': 'Invoice', 'documentNumber': '2022-09741'}, 'keyTuple': ('2022-09741', '2022'), 'paymentList': [{'documentNumber': '2022-09741', 'paymentDate': '2022-08-21', 'valuePaid': 390, 'documentCurrency': 'EUR'}]}


In [44]:
def mergerFunction(leftDict,rightDict):
  leftDict["paymentList"] +=rightDict["paymentList"]
  return leftDict
  
reducedNiceJoinedRDD = niceJoineRDD.map(lambda row: (row.get("keyTuple"),row)).reduceByKey(lambda left,right: mergerFunction(left,right) ).map(lambda x: x[1])
reducedNiceJoinedRDD.first()

{'header': {'customerId': 'Customer_7503',
  'value': 1993,
  'documentCurrency': 'EUR',
  'postingDate': '2022-04-01',
  'dueDate': '2022-05-23',
  'fiscalYear': '2022',
  'documentType': 'Invoice',
  'documentNumber': '2022-04514'},
 'keyTuple': ('2022-04514', '2022'),
 'paymentList': [{'documentNumber': '2022-04514',
   'paymentDate': '2022-06-26',
   'valuePaid': 1560,
   'documentCurrency': 'EUR'}]}

In [46]:
reducedNiceJoinedRDD.filter(lambda row: row.get("keyTuple")==('2022-09741', '2022')).first()

{'header': {'customerId': 'Customer_2351',
  'value': 3930,
  'documentCurrency': 'EUR',
  'postingDate': '2022-07-15',
  'dueDate': '2022-08-03',
  'fiscalYear': '2022',
  'documentType': 'Invoice',
  'documentNumber': '2022-09741'},
 'keyTuple': ('2022-09741', '2022'),
 'paymentList': [{'documentNumber': '2022-09741',
   'paymentDate': '2022-09-14',
   'valuePaid': 882,
   'documentCurrency': 'EUR'},
  {'documentNumber': '2022-09741',
   'paymentDate': '2022-08-21',
   'valuePaid': 390,
   'documentCurrency': 'EUR'}]}

# Alternative and scalable approach

In [48]:
def quantitativeRepr(row):
  return {"amount":row.get("valuePaid"),"numberOfPayments":1,"lastDate":row.get("paymentDate")}

def combineFun(firstPayment,secondPayment):
  firstPayment["amount"] += secondPayment.get("amount")
  firstPayment["numberOfPayments"] += secondPayment.get("numberOfPayments")
  firstPayment["lastDate"] = secondPayment.get("lastDate") if secondPayment.get("lastDate")>firstPayment.get("lastDate") else firstPayment.get("lastDate")
  return firstPayment
  
lightPaymentRDD = paymentsRDD.map(lambda x: (x.get("documentNumber"),quantitativeRepr(x))) \
                .reduceByKey(lambda firstPayment,secondPayment: combineFun(firstPayment,secondPayment)) \
                .map(lambda x: {"keyTuple":x[0],"paymentStats":x[1]})
lightPaymentRDD.collect()

[{'keyTuple': '2022-06524',
  'paymentStats': {'amount': 796,
   'numberOfPayments': 1,
   'lastDate': '2022-08-01'}},
 {'keyTuple': '2022-09999',
  'paymentStats': {'amount': 1244,
   'numberOfPayments': 2,
   'lastDate': '2022-08-27'}},
 {'keyTuple': '2022-02879',
  'paymentStats': {'amount': 4108,
   'numberOfPayments': 3,
   'lastDate': '2022-05-08'}},
 {'keyTuple': '2022-03091',
  'paymentStats': {'amount': 2131,
   'numberOfPayments': 1,
   'lastDate': '2022-04-24'}},
 {'keyTuple': '2022-05054',
  'paymentStats': {'amount': 651,
   'numberOfPayments': 1,
   'lastDate': '2022-06-11'}},
 {'keyTuple': '2022-02608',
  'paymentStats': {'amount': 16658,
   'numberOfPayments': 3,
   'lastDate': '2022-05-10'}},
 {'keyTuple': '2022-02287',
  'paymentStats': {'amount': 773,
   'numberOfPayments': 1,
   'lastDate': '2022-03-18'}},
 {'keyTuple': '2022-00308',
  'paymentStats': {'amount': 3287,
   'numberOfPayments': 1,
   'lastDate': '2022-01-22'}},
 {'keyTuple': '2022-01040',
  'paymentStat

In [50]:
#lightPaymentRDD.map(lambda x: (x.get("keyTuple"),x)).join(semiJoinHeaderRDD).first()

In [0]:
semiJoinHeaderRDD.union(sc.parallelize(["pippo"])).filter(lambda row: type(row)==tuple and type(row[1])==dict).join(semiJoinPaymentsRDD).count()

Out[34]: 12500

In [0]:
#myString = "a1"
#myString[0],myString[1]

Out[32]: ('a', '1')

In [51]:
#sc.parallelize(["a1","a2","a1","b1","b2"]).reduceByKey(lambda first,second: first+second).collect()

[('a', '121'), ('b', '12')]

# Part 02
* How many invoices are open (i.e., not completely paid)?
* How many invoices are closed (i.e., completely paid)?
* How many invoices are overdued (i.e., not completely paid and with a due date in the past)?
* How many invoices have been paid not in time (i.e., completely paid and with the last payment after the due date)?
* Add to the RDD the information of "closingDate" as the date of the payment that close that invoice.
* Add to the RDD the boolean of "inTime": True if the closingDate < dueDate else False

# Part 03 - Debit Note
* How many invoices have been paid for more then their value?
* Add to the Header RDD for each of them a Debit Note with the value to be charged back and the date of today

# Part 04 - Paymenets Frequency
* Add to the Payment Rdd the computed "expectedPaymentDate". It is based on the two previous payments, and is the last payment date + the difference between it and the payment right before, customer by customer.
So, in the example below, for the first two payment is not possible to compute, while for the third, the expected payment is the 2022/10/15 (date of the last payment) plus 3 (the difference between it and the payment of 2022/10/12) 
| customerId  | paymentDate | expectedPaymentDate | documentNumber | ... |
|-------------|-------------|---------------------|----------------|-----|
| Customer001 | 2022/10/12  | N/A                 | 2022_01001     | ... |
| Customer001 | 2022/10/15  | N/A                 | 2022_01004     | ... |
| Customer001 | 2022/10/16  | 2022/10/18 (15+3)   | 2022_00904     | ... |
| Customer001 | 2022/10/20  | 2022/10/17 (16+1)   | 2022_01004     | ... |
| Customer001 | 2022/10/30  | 2022/11/24 (20+4)   | 2022_01101     | ... |
| Customer001 | ...         | ...                 | ...            | ... |
* Show for each customer, the average error of such method

# Part 05 - Cosine Similarity
* How many customers has the company?
* Draw the histogram - without using .hist() - as the number of customer with 1 invoice, the number of customers with 2 invoices, ...
* Define two customers similarity based on the cosine similarity computed on the average payment time per day
    * a day with no invoice posted count as zero
    * for other days, compute the average payment timing using the due date as zero (10 days in advance means -10, 3 days after means +3)